In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import imodelsx.process_results
import sys
import datasets
import numpy as np
import clin.eval

sys.path.append('../experiments/')
results_dir = '../results/'

In [ ]:
r = imodelsx.process_results.get_results_df(results_dir, use_cached=True)

# get data for eval
dset = datasets.load_dataset('mitclinicalml/clinical-ie', 'medication_status')
df_val = pd.DataFrame.from_dict(dset['validation'])
df = pd.DataFrame.from_dict(dset['test'])
# df = pd.concat([val, test])
nums = np.arange(len(df)).tolist()
np.random.default_rng(seed=13).shuffle(nums)
dfe = df.iloc[nums]

# add medication status eval
accs_cond, f1s_macro_cond = clin.eval.eval_medication_status(dfe, r)
r['acc_cond'] = accs_cond
r['f1_macro_cond'] = f1s_macro_cond

In [ ]:
(
    r.groupby(['checkpoint', 'n_shots'])[['f1', 'recall', 'precision', 'acc_cond', 'f1_macro_cond']].mean()
    .style.format(precision=3).background_gradient(cmap='Blues')
)

# Test verifiers

In [ ]:
import clin.verifiers.evidence
import clin.llm
import clin.parse
import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'

In [199]:
row = r[(r.n_shots == 5) * (r.checkpoint == 'text-davinci-003')].iloc[0]
llm = clin.llm.get_llm('text-davinci-003')
meds_with_evidence_dicts = []
ev = clin.verifiers.evidence.EvidenceVerifier()
resps = row['resps']

for i in tqdm(range(len(resps))):
    resp = resps[i]
    snippet = dfe.iloc[i]['snippet']

    meds_with_evidence_dicts.append(ev(snippet=snippet, bulleted_str=resp, llm=llm))


 17%|█▋        | 17/100 [00:13<01:48,  1.31s/it]

In [ ]:
for d in meds_with_evidence_dicts:
    for k in d:
        print(k, d[k])
    print()

ativan ('active', 'ativan 1 mg p.o. q. 8 hours and use ativan 1 mg iv q. 4 hours p.r.n.')
inderal la ('active', 'inderal la 60 mg p.o. q.d.')
celexa ('discontinued', 'she does not want to take celexa')
lexapro ('active', 'lexapro 2 mg p.o. q.d.')

coumadin ('active', 'continue coumadin anticoagulation')
vistaril ('discontinued', 'dc vistaril')
percocet ('discontinued', 'change percocet to oxycodone')
oxycodone ('active', 'oxycodone 5 mg 1-2 tablets q.3h p.r.n.')

ditropan ('discontinued', 'the patient had been treated with ditropan for bladder spasms, but has not needed this for the entire past week')
cidofovir ('active', 'the patient has received five doses of cidofovir and will receive her last on _%#mmdd2003#%_, as described earlier')

zosyn ('active', "given the patient's bilateral pleural effusions and increased left-sided infiltrate, he was treated for hospitalacquired pneumonia empirically with zosyn")
levofloxacin ('active', "given the patient's bilateral pleural effusions and 